In [ ]:
# Bibliotheken importieren
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
import os
from symspellpy import SymSpell, Verbosity
from importlib.resources import files, as_file

In [ ]:
# Verzeichnisse für Modelle und Plots erstellen, falls sie nicht existieren
os.makedirs('../models', exist_ok=True)
os.makedirs('../results', exist_ok=True)

In [ ]:
# Download für englischensprachigen Datensatz
!python -m spacy download en_core_web_sm

In [ ]:
# Initialisierungen & Downloads
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')
    nlp = spacy.load('en_core_web_sm')
    print("NLP-Bibliotheken erfolgreich initialisiert.")
except Exception as e:
    print("Fehler beim Laden der NLP-Komponenten:")
    print(e)

# Spacy-Modell laden
nlp = spacy.load('en_core_web_sm')

# Prüfen ob der Datensatz vorhanden ist und laden
try: 
    # Pfad ggf. anpassen
    data = pd.read_csv('../data/consumer_complaints.csv', low_memory=False)
    print(f"Datensatz vorhanden und geladen: {data.shape[0]} Zeilen und {data.shape[1]} Spalten")
except FileNotFoundError:
    print("Datei nicht gefunden. Bitte lade den Datensatz herunter und speichere ihn im 'data'-Ordner.")

In [ ]:
# SymSpell initialisieren, https://github.com/wolfgarbe/SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Lade die Ressource aus dem symspellpy-Paket
# Pfad zur Wörterbuch-Datei holen
with as_file(files("symspellpy") / "frequency_dictionary_en_82_765.txt") as dictionary_path:
    sym_spell.load_dictionary(str(dictionary_path), term_index=0, count_index=1)


In [ ]:
# Zeigt die ersten 5 Zeilen des DataFrames an (Vorschau auf den Datensatz)
data.head()

In [ ]:
# Zeigt die Struktur des DataFrames: Spaltennamen, Datentypen und Anzahl der Nicht-Null-Werte
data.info()

In [ ]:
# Zeigt statistische Kennzahlen für numerische Spalten (z. B. Mittelwert, Standardabweichung, Min/Max)
data.describe()

In [ ]:
# Zählt die Anzahl der fehlenden (NaN) Werte in jeder Spalte
data.isnull().sum()

In [ ]:
# Prüft, ob die Spalte 'consumer_complaint_narrative' im DataFrame existiert
if 'consumer_complaint_narrative' in data.columns:
    # Filtert nur die Zeilen, in denen ein Beschwerdetext vorhanden ist (ohne NaN)
    complaints = data[data['consumer_complaint_narrative'].notna()]['consumer_complaint_narrative']
    # Gibt die Anzahl der Beschwerden mit Text aus
    print(f"Anzahl der Beschwerden mit Text: {len(complaints):,} von {len(data):,}".replace(",","."))
else:
    print("Spalte mit Beschwerdetexten nicht gefunden. Überprüfe die Spaltennamen:")
    print(data.columns.tolist())

In [ ]:
# Funktion zur Textvorverarbeitung
def preprocess_text(text):
    # Prüfen ob text ein string ist
    if not isinstance(text, str):
        return ""

    # 1. Kleinbuchstaben
    text = text.lower()

    # 2. Entfernen von Sonderzeichen und Zahlen (nur Buchstaben und Leerzeichen behalten)
    text = re.sub(r'[^a-z\s]', '', text) # Nur Kleinbuchstaben und Leerzeichen beibehalten, wegen ^ nicht diese Zeichen entfernen !!!

    # 3. Tokenisierung (Text in Wörter zerlegen)
    tokens = word_tokenize(text, preserve_line=True)

    # 4. Stoppwörter entfernen (häufige Wörter wie 'the', 'is', 'in')
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatisierung (Wörter auf ihre Grundform zurückführen, z.B. 'running' -> 'run')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 5.5 Entfernen von Mustern wie 'xx', 'xxx', 'xxxx' usw. | Dieser Schritt ist erst nach der Erstellung der ersten Tabelle mit TF-IDF aufgefallen.
    # re.fullmatch prüft das ganze wort auf das Muster
    tokens = [word for word in tokens if not re.fullmatch(r'x{2,}', word)]

    # 5.6 : Entfernen von Wörtern, die x-Sequenzen enthalten (z.B. "xxxx1234" oder "xxxxbank")
    tokens = [word for word in tokens if not re.search(r'x{2,}', word)]

    # 6. Entfernen von sehr kurzen Wörtern (oft Rauschen)
    tokens = [word for word in tokens if len(word) > 2]

    # 7. Rechtschreibkorrektur mit SymSpellPy für allgemeines Englisch
    corrected_tokens = []
    for word in tokens:
        # Nur für Wörter mit mindestens 3 Buchstaben die Rechtschreibkorrektur anwenden
        if len(word) <= 2:
            corrected_tokens.append(word)
            continue
            
        # SymSpell für allgemeine Rechtschreibkorrektur verwenden
        # https://symspellpy.readthedocs.io/en/latest/api/symspellpy.html
        try:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            if suggestions:
                # Wähle die wahrscheinlichste Korrektur, wenn das Konfidenzmaß hoch genug ist
                if suggestions[0].distance <= 1:  # Nur Korrekturen mit geringer Edit-Distanz
                    corrected_tokens.append(suggestions[0].term)
                else:
                    corrected_tokens.append(word)  # Original behalten wenn unsicher
            else:
                corrected_tokens.append(word)
        except:
            corrected_tokens.append(word)

    # 8. Entfernen von sehr kurzen Wörtern (oft Rauschen)
    corrected_tokens = [word for word in corrected_tokens if len(word) > 2]

    # Rückgabe als einzelner String mit Leerzeichen getrennt
    return ' '.join(corrected_tokens)

In [ ]:
# Vorverarbeitung auf einer kleinen Stichprobe testen
print("\nTest der Vorverarbeitung an Beispielen:")
sample = complaints.sample(5)
for i, text in enumerate(sample):
    print(f"Original {i+1}: {text[:100]}...")
    print(f"Verarbeitet {i+1}: {preprocess_text(text)[:100]}...")
    print("-" * 80)

In [ ]:
# Vorverarbeitung auf alle Texte anwenden (kann etwas dauern | bei mir ca. 3 Minuten))
# Hinweis: Bei sehr großen Datensätzen kann dies länger dauern.
processed_complaints = complaints.apply(preprocess_text)

# Speichern für die nächste Phase
processed_df = pd.DataFrame({'processed_text': processed_complaints})
processed_df.to_csv('../data/processed_complaints_v2.csv', index=False)